# Introductive code

In order to work properly, our notebook requires 2 folders:

+ "MaskDataset" folder, which is the result of unzipping 'artificial-neural-networks-and-deep-learning-2020.zip': we've created a cell that unzips the file, but it should be run only if "MaskDataset" folder wasn't already created

+ "MaskDatasetSorted" folder, which contains three folders (one per label) with all the images for flow_from_directory: it can be created running the three code cells below the "Sort the images for flow_from_directory method" title, and each of them should be run only 1 time (in order to avoid multiple copies of the same image)

In [ ]:
# Print all the intermediate operations (for debugging)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Import most relevant libraries

import os
import tensorflow as tf
import numpy as np


# Import library for handling json files

import json


# Import shutil for the copy in sorted folders

import shutil

In [ ]:
# Fix a seed

SEED = 1234
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [ ]:
# Add Colab (with Drive)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# If the "MaskDataset" folder wasn't already created, this cell unzips the zip file and creates "MaskDataset";
# please run this cell only if "MaskDataset" wasn't already created

!unzip '/content/drive/My Drive/ANNDL_Results/Challenge1/artificial-neural-networks-and-deep-learning-2020.zip' -d '/content/drive/My Drive/ANNDL_Results/Challenge1'

In [ ]:
# Path to the current working directory (which contains MaskDataset folder)

cwd = '/content/drive/My Drive/ANNDL_Results/Challenge1' # Working directory, with the dataset folder (we had issues with os.getcwd())

# The following variables are inherited from previous code for Kaggle: they may be 
# changed to switch to Kaggle without changing the rest of the implementation

work_dir = cwd
work_dir2 = cwd
work_dir3 = cwd

In [ ]:
# Path to the dataset folder (MaskDataset folder must be already created)

dataset_dir = os.path.join(work_dir,"MaskDataset")

# Sort the images for flow_from_directory method

Run the following 3 cells to create "MaskDatasetSorted" folder, which contains three folders (one per label) with all the images for flow_from_directory. Please run these 3 cells only if "MaskDatasetSorted" wasn't already created

In [ ]:
# From the file .json, extract a dictionary with:

# Key: name of the image
# Value: label of the image

with open(os.path.join(dataset_dir,"train_gt.json")) as json_file: 
    labels = json.load(json_file)

In [ ]:
# Create the folders for flow_from_directory method

sorted_training_dir = os.path.join(work_dir2,"MaskDatasetSorted")
if not os.path.exists(sorted_training_dir):
    os.makedirs(sorted_training_dir)

folder_0 = os.path.join(sorted_training_dir,"label0")
if not os.path.exists(folder_0):
    os.makedirs(folder_0)
    
folder_1 = os.path.join(sorted_training_dir,"label1")
if not os.path.exists(folder_1):
    os.makedirs(folder_1)

folder_2 = os.path.join(sorted_training_dir,"label2")
if not os.path.exists(folder_2):
    os.makedirs(folder_2)

In [ ]:
# Copy the images in the proper folder

training_dir = os.path.join(dataset_dir, 'training')

for name in labels.keys():
    photo_dir = os.path.join(training_dir,name)
    
    if labels[name] == 0:
        shutil.copy2(photo_dir,folder_0)
    if labels[name] == 1:
        shutil.copy2(photo_dir,folder_1)
    if labels[name] == 2:
        shutil.copy2(photo_dir,folder_2)

# Set all the necessary variables in ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object (for data augmentation)

if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=0,
                                        height_shift_range=0,
                                        zoom_range=0.15,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255,
                                        validation_split = 0.08)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255,
                                        validation_split = 0.08)

# Create train and validation generators

In [ ]:
# Path to the training folder (data will be splitted, using ImageDataGenerator objects with flow_from_directory method)

training_dir = os.path.join(work_dir2, 'MaskDatasetSorted')

In [ ]:
# Some useful parameters

# Batch size
bs = 32

# img shape
img_h = 256
img_w = 256

# Number of classes
num_classes = 3

In [ ]:
# Create generators to read images

decide_class_indices = True
if decide_class_indices:
    classes = ['label0',    # 0
               'label1',    # 1
               'label2']    # 2
else:
    classes=None

# Training
    
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size=(img_h, img_w),
                                               color_mode="rgb",
                                               classes=classes,
                                               class_mode='categorical',
                                               batch_size=bs,
                                               shuffle=True,
                                               seed=SEED,
                                               subset='training')

# Validation

valid_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size=(img_h, img_w),
                                               color_mode="rgb",
                                               classes=classes,
                                               class_mode='categorical',
                                               batch_size=bs,
                                               shuffle=True,
                                               seed=SEED,
                                               subset='validation')

Found 5166 images belonging to 3 classes.
Found 448 images belonging to 3 classes.


In [ ]:
# Check how keras assigned the labels
train_gen.class_indices

{'label0': 0, 'label1': 1, 'label2': 2}

In [ ]:
# Create Dataset objects both for training and for validation with tf.data.Dataset.from_generator


# Training

train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()


# Validation

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
valid_dataset = valid_dataset.repeat()

# Build and fit the model

In [ ]:
# Set the model

start_f = 32
depth = 5

model = tf.keras.Sequential()

# Perform features extraction with convolutional layers

for i in range(depth):

    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape=[None]

    # Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

    start_f *= 2
    
# Set the fully connected layers for classification

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(units=32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
# Visualize created model as a table

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
re_lu (ReLU)                 (None, 256, 256, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       7

In [ ]:
# Optimization params

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()


# Learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)


# Validation metrics
metrics = ['accuracy']


# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

We trained the model for several times running the *model.fit* cell many times (in order to check the validity of the model). You can see below the result of the last 5 epochs (being novice, we overwrote the previous results). Our checkpoint is one of the results of this approach. The following cell is taken from the lab session, saves checkpoints and performs early stopping. If you don't want checkpoints, please comment the line '*callbacks.append(ckpt_callback)*'. To replicate our model, please run *model.fit* with around 65 epochs.

In [ ]:
import os
from datetime import datetime

cwd = cwd  # use your local directory if you are not using Drive

exps_dir = os.path.join(cwd, 'classification_experiments_')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

exp_name = 'Model_3'

exp_dir = os.path.join(exps_dir, exp_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), save_best_only=True,
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Early Stopping

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)

In [ ]:
# Fit the model (few epochs because of problems with checkpoints)

model.fit(x=train_dataset,
          epochs=5,  
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Epoch 1/5
162/162 [==============================] - 98s 605ms/step - loss: 0.2555 - accuracy: 0.9139 - val_loss: 0.5205 - val_accuracy: 0.8080
Epoch 2/5
162/162 [==============================] - 98s 607ms/step - loss: 0.2528 - accuracy: 0.9139 - val_loss: 0.4834 - val_accuracy: 0.8036
Epoch 3/5
162/162 [==============================] - 98s 603ms/step - loss: 0.2559 - accuracy: 0.9117 - val_loss: 0.4482 - val_accuracy: 0.8147
Epoch 4/5
162/162 [==============================] - 99s 609ms/step - loss: 0.2483 - accuracy: 0.9185 - val_loss: 0.4429 - val_accuracy: 0.8415
Epoch 5/5
162/162 [==============================] - 98s 608ms/step - loss: 0.2392 - accuracy: 0.9202 - val_loss: 0.5417 - val_accuracy: 0.8013


In [ ]:
model.load_weights(os.path.join('/content/drive/My Drive/ANNDL_Results/Challenge1/Our_best_model_weights', 'BestCheckpoint.ckpt'))

# Test the model, generate the csv file

In [ ]:
# Import necessary libraries

import os

from datetime import datetime

from PIL import Image

In [ ]:
# Given function for saving the csv file, once the experiment is complete

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
# our version

test_dir = os.path.join(dataset_dir, 'test')
image_filenames = next(os.walk(test_dir))[2]

results = {}

# Iterate on each image
for image_name in image_filenames:

   # Open image
   img = Image.open(os.path.join(test_dir, image_name)).convert('RGB')

   # Create a tensor from each image

   img_array = np.array(img.resize((img_w,img_h)))
   img_array = np.expand_dims(img_array, 0) 

   # Normalize, predict and add to the dictionary
   softmax = model.predict(x = (img_array/255.))
   prediction = tf.argmax(softmax,1)
   results[image_name] = int(prediction)

# Create csv file with the given function
create_csv(results,work_dir3)